In [1]:
#!pip install --no-cache-dir -I pillow
!pip install hdf5storage
#!pip install http://download.pytorch.org/whl/cu92/torch-1.6.0-cp36-cp36m-linux_x86_64.whl
#!pip install torch
!pip3 install torchvision
#!git clone https://github.com/lanpa/tensorboardX && cd tensorboardX && python setup.py install
!pip install tensorboardX
!pip install --pre torch -f  https://download.pytorch.org/whl/nightly/cu101/torch-1.7.0.dev20200626%2Bcu101-cp36-cp36m-linux_x86_64.whl


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 988 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 9.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/nightly/cu101/torch-1.7.0.dev20200626%2Bcu101-cp36-cp36m-linux_x86_64.whl


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
#os.chdir("/content/drive/My Drive/maize_season3_RededgeMultispectral_20181030_10m_flight2/") # folder containing your images used for training and testing 
#os.chdir("/content/drive/My Drive/maize_season3_RededgeMultispectral_20181119_10m/") # folder containing your images used for training and testing 
os.chdir("/content/drive/My Drive/")
path=os.getcwd() 

Mounted at /content/drive


In [3]:
### 
from __future__ import division
from scipy import interpolate
import random
import os
import os.path
import h5py
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import PIL

import scipy.io
import matplotlib
matplotlib.use('Agg')
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
from IPython.display import clear_output 

import torchvision.utils as utils
from tensorboardX import SummaryWriter
###
import torch
import torch.nn as nn
import argparse
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.utils.data as udata
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.utils as utils
import time
import scipy.io as sio
import logging
import hdf5storage
import datetime
from math import sqrt
%matplotlib inline
import scipy.io as spio
from scipy.interpolate import PchipInterpolator
from bisect import bisect
from google.colab import output
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pylab import cm
##

class AverageMeter(object):
    """Computes and stores the average and current value."""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, evaluation):
    """Save the checkpoint."""
    state = {
            'epoch': epoch,
            'iter': iteration,
            'state_dict': model.state_dict(),
            'optimizer' : optimizer.state_dict(),
            }
    
    torch.save(state, os.path.join(model_path, 'hscnn_6layer_dim10_{}.pkl'.format(evaluation)))

def plot_spectrum(real, fake, epoch, i):
    x =np.linspace(400, 900, 5, endpoint=True) # the wavebands of the hyperspectral image
    fig = Figure()
    canvas = FigureCanvasAgg(fig)
    ax = fig.gca()
    #ax.set_ylim(0, 1)
    plot_real,  = ax.plot(x, real, 'ko-')
    plot_fake,  = ax.plot(x, fake, 'r.-')
    fig.legend((plot_real,plot_fake), ('real', 'fake'))
    canvas.draw()
    fig.savefig(os.path.join(iteration_path, "{}_test_{}.png".format(epoch,i)))
    I = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    I = I.reshape(canvas.get_width_height()[::-1]+(3,))
    I = np.transpose(I, [2,0,1])
    return np.float32(I)


def plotwithcolorbar(img, title=None, figsize=(10,10)):
    ''' Plot an image with a colorbar '''
    vmin = np.min(img)
    vmax = np.max(img)
    hh = img.shape[1]
    fig, axis = plt.subplots(1, 1, figsize=figsize)
    rad2 = axis.imshow(img, vmin=vmin, vmax=vmax)
    axis.set_title(title)
    divider = make_axes_locatable(axis)
    cax = divider.append_axes("right", size="3%", pad=0.05)
    fig.colorbar(rad2, cax=cax)
    circle_b = plt.Circle((int(hh/2), int(hh/2)), 5, color='b', fill=False)
    axis.add_artist(circle_b)
    plt.close(fig)
    #fig.savefig(os.path.join(iteration_path, "{}_test_fake_img_{}.png".format(epoch,i)))
    #plt.tight_layout()
    #plt.show()
    return fig, axis

def initialize_logger(file_dir):
    """Print the results in the log file."""
    logger = logging.getLogger()
    fhandler = logging.FileHandler(filename=file_dir, mode='a')
    formatter = logging.Formatter('%(asctime)s - %(message)s',"%Y-%m-%d %H:%M:%S")
    fhandler.setFormatter(formatter)
    logger.addHandler(fhandler)
    logger.setLevel(logging.INFO)
    return logger

def save_matv73(mat_name, var_name, var):
    hdf5storage.savemat(mat_name, {var_name: var}, format='7.3', store_python_metadata=True)


def mrae_loss(im_true, im_fake):
    error = torch.abs(im_fake-im_true)/im_true
    rrmse = torch.mean(error.reshape(-1))
    return rrmse

def sam_loss(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    nom = torch.sum( torch.mul(im_true, im_fake), dim=1)
    denom1 = torch.sqrt( torch.sum( torch.pow(im_true,2), dim=1))
    denom2 = torch.sqrt( torch.sum( torch.pow(im_fake,2), dim=1))
    sam = torch.acos(torch.div(nom, torch.mul(denom1, denom2)).clamp(-1.0 + 1e-8, 1.0 - 1e-8))
    #print(sam.size())
    sam = torch.mul(torch.div(sam, np.pi), 180)
    #print(("sam_max", sam.max()))
    #print(("sam_min", sam.min()))
    sam = torch.div(torch.sum(sam), N*H*W)
    #print("")
    #print(("sam_final", sam.item()))
    #print("")
    return sam

def sid_loss(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    denom1 = torch.sqrt( torch.sum( torch.pow(im_true,2), dim=1))
    denom2 = torch.sqrt( torch.sum( torch.pow(im_fake,2), dim=1))
    #
    unit_t = torch.div(im_true, denom1.unsqueeze(1))
    uint_f = torch.div(im_fake, denom2.unsqueeze(1))
    #
    sid = ((unit_t - uint_f)* (unit_t.log() - uint_f.log())).sum() / (N*H*W)
    return sid
####
## new fucntion for process data
path=os.getcwd() 

##. with normalization
def normalize(data):
    #data_nl = (data-np.amin(data[:,:,:]))/(np.amax(data[:,:,:])-np.amin(data[:,:,:]))
    data_nl = data/np.amax(data)
    #data_nl[data_nl<=0.0] = 0.0
    #data_nl[data_nl>1.0] = 1.0
    return data_nl
##
def gen_random_scale_n(img, rnd=3):
    np.random.seed(rnd)
    scale = np.random.uniform(0.1, 1.91, (1,1, img.shape[2],img.shape[3]))   
    return img*scale
##
def data_process_list_n(path=path):
    NO_ = 1
    hyper_f = os.path.join(path,'Maize2018_ortho_msi_cropped_04052021')
    sub_h5_fd = next(os.walk(hyper_f))[1]
    sub_h5_fd.sort()
    #
    #rgb_f = hyper_f.replace("h5", "rgb")
    rgb_f = os.path.join(path,'Maize2018_ortho_rgb_cropped_04052021_NEW')
    sub_rgb_fd = next(os.walk(rgb_f))[1]
    sub_rgb_fd.sort()

    for sf in range(len(sub_h5_fd)):
    #for sf in [0,1,2,3]:
        id_n = sub_h5_fd[sf].split("_")[2][4:]
        #
        filenames_hyper = glob.glob(os.path.join(hyper_f,sub_h5_fd[sf],'*.h5'))
        filenames_rgb = glob.glob(os.path.join(rgb_f, sub_rgb_fd[sf].replace("h5", "rgb"),'*.png'))
        filenames_hyper.sort()
        print(filenames_hyper)
        filenames_rgb.sort()
        print(filenames_rgb)
        #h5f = h5py.File('train_uavf3.h5', 'w')  ## the file concatenated hsi and rgb in one file
        h5f = h5py.File('train_040521_maize_NatColor_finalN_{}.h5'.format(id_n), 'w')  ## the file concatenated hsi and rgb in one file
        #filenames_hyper = glob.glob(os.path.join(path,'Rice_ortho_h5_cropped_noco/Exported_rice_20181124_h5_cropped_2','*.h5'))
        
        
        for i in range(len(filenames_hyper)):
            print("\n")
            print(filenames_hyper[i], filenames_rgb[i])
            # load hyperspectral image
            mat =  h5py.File(filenames_hyper[i],'r')
            hyper = np.float32(np.array(mat['img']))
            print(("hyper_shape",hyper.shape))
            hyper = np.transpose(hyper, [2,0,1])#/32768.0 # because it 32768.0 was divided already before cropping into 512x512 squares
            #print("hyper_or_max", np.amax(hyper))
            #print("hyper_or_min",np.amin(hyper))
            #hyper[hyper<0] = 0
            #hyper[hyper>1.0] = 1.0
            #hyper = normalize(hyper)
            print("hyper_test_max", np.amax(hyper))
            print("hyper_test_min",np.amin(hyper))
            mat.close()
            # load rgb image
            rgb =  cv2.imread(filenames_rgb[i])
            rgb=cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)

            rgb = np.float32(np.transpose(rgb, [2,0,1])/255.0) # change to 0-1 range
            #print("rgb_or_max", np.amax(np.float32(rgb)))
            #print("rgb_or_min", np.amin(np.float32(rgb)))
            #rgb = normalize(np.float32(rgb))
            print("rgb_test_max", np.amax(np.float32(rgb)))
            print("rgb_test_min", np.amin(np.float32(rgb)))
    #            mat.close()
                # creat patches
            data = np.concatenate((hyper,rgb), 0)
            h5f.create_dataset(str(NO_), data=data)
            NO_ += 1
        h5f.close()
        print("NO. of samples: {}".format(NO_-1))
##
class HyperDataset_list_maize_RGB(udata.Dataset):
    def __init__(self, crop_size=64):
        self.crop_size = crop_size
        xx = ["train_040521_maize_NatColor_finalN_1030.h5","train_040521_maize_NatColor_finalN_1109.h5", "train_040521_maize_NatColor_finalN_1119.h5","train_040521_maize_NatColor_finalN_1220.h5"] #"train_uavf_maize_NatColor_1109.h5" was removed because of over exposure
        xx.sort()
        # print(xx)
        key_is = []
        for i in range(len(xx)):
            f_i = xx[i]
            h5f_i = h5py.File(f_i, 'r')
            keys_i = list(h5f_i.keys())
            keys_i.sort()
            key_i_n  = ["_".join([str(i), x]) for x in keys_i]
            key_is = key_is + key_i_n
            #print(("key_is", key_is))
            h5f_i.close()
        
        self.keys = key_is
        
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        #xx = glob.glob(os.path.join(path,'train_uavf_maize_natlk_2_shuffle_?*.h5'))
        xx = ["train_040521_maize_NatColor_finalN_1030.h5","train_040521_maize_NatColor_finalN_1109.h5", "train_040521_maize_NatColor_finalN_1119.h5","train_040521_maize_NatColor_finalN_1220.h5"] #"train_uavf_maize_NatColor_1109.h5" was removed because of over exposure
        xx.sort()
        key = str(self.keys[index])
        #print(("key", key))
        key_path_i = key.split("_")[0]
        #print(("key_path_i",key_path_i))
        key_key = key.split("_")[1]
        #print(("key_key",key_key))
        #print(xx[int(key_path_i)])
        h5f_a = h5py.File(xx[int(key_path_i)], 'r')
        #
        #print(("list",list(h5f_a.keys())))
        #print(key_key)
        data_a = np.array(h5f_a[key_key])
        data = torch.as_tensor(data_a)
        #data[data>1.0] = 1.0
        #data[data<0.0] = 0.0
        #print(data.size())
        # crop
        w = int(data.size()[1])
        h = int(data.size()[2])
        th, tw = self.crop_size, self.crop_size
        if w > tw or h > th:
            i = 0
            j = 0
            data = data[:,i:i+th,j:j+tw]
        h5f_a.close()
        return data[0:5,:,:], data[5:8,:,:]

class HyperDataset_list_rice_RGB(udata.Dataset):
    def __init__(self, crop_size=64):
        self.crop_size = crop_size
        #xx = glob.glob(os.path.join(path,'train_uavf_maize_natlk_2_shuffle_?*.h5'))
        xx = ["train_uavf_rice_NatColor_final_0821.h5","train_uavf_rice_NatColor_final_1026.h5","train_uavf_rice_NatColor_final_1124.h5"] #"train_uavf_rice_NatColor_final_0927.h5" was removed due to over exposure
        xx.sort()
        # print(xx)
        key_is = []
        for i in range(len(xx)):
            f_i = xx[i]
            h5f_i = h5py.File(f_i, 'r')
            keys_i = list(h5f_i.keys())
            keys_i.sort()
            key_i_n  = ["_".join([str(i), x]) for x in keys_i]
            key_is = key_is + key_i_n
            #print(("key_is", key_is))
            h5f_i.close()
        
        self.keys = key_is
        
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        #xx = glob.glob(os.path.join(path,'train_uavf_maize_natlk_2_shuffle_?*.h5'))
        xx = ["train_uavf_rice_NatColor_final_0821.h5","train_uavf_rice_NatColor_final_1026.h5","train_uavf_rice_NatColor_final_1124.h5"] #"train_uavf_rice_NatColor_final_0927.h5" was removed due to over exposure
        xx.sort()
        key = str(self.keys[index])
        #print(("key", key))
        key_path_i = key.split("_")[0]
        #print(("key_path_i",key_path_i))
        key_key = key.split("_")[1]
        #print(("key_key",key_key))
        #print(xx[int(key_path_i)])
        h5f_a = h5py.File(xx[int(key_path_i)], 'r')
        #
        #print(("list",list(h5f_a.keys())))
        #print(key_key)
        data_a = np.array(h5f_a[key_key])
        data = torch.as_tensor(data_a)
        #data[data>1.0] = 1.0
        #data[data<0.0] = 0.0
        #print(data.size())
        # crop
        w = int(data.size()[1])
        h = int(data.size()[2])
        th, tw = self.crop_size, self.crop_size
        if w > tw or h > th:
            i = 0
            j = 0
            data = data[:,i:i+th,j:j+tw]
        h5f_a.close()
        return data[0:5,:,:], data[5:8,:,:]

def batch_MRAE(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).reshape(N, C*H*W)
    mse = nn.MSELoss(reduction='none')
    err = mse(Itrue, Ifake).sqrt_().div_(Itrue).sum(dim=1, keepdim=True).div_(C*H*W)
    return torch.mean(err)

def batch_RMSE(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clamp(0.,1.).reshape(N, C*H*W)
    Ifake = im_fake.clamp(0.,1.).reshape(N, C*H*W)
    mse = nn.MSELoss(reduction='none')
    err = mse(Itrue, Ifake).sum(dim=1, keepdim=True).div_(C*H*W).sqrt_()
    return torch.mean(err)

def batch_SID(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clone().reshape(N, C, H*W)
    Ifake = im_fake.clone().reshape(N, C, H*W)
    #nom = torch.mul(Itrue, Ifake).sum(dim=1).reshape(N, H*W)
    denom1 = torch.pow(Itrue,2).sum(dim=1).sqrt_().reshape(N, H*W)
    denom2 = torch.pow(Ifake,2).sum(dim=1).sqrt_().reshape(N, H*W)
    #
    unit_t = torch.div(Itrue, denom1.unsqueeze(1))
    uint_f = torch.div(Ifake, denom2.unsqueeze(1))
    #
    sid = ((unit_t - uint_f)* (unit_t.log() - uint_f.log())).sum(dim = 1).reshape(N, H*W)
    sid = sid.sum() / (N*H*W)
    return sid


def batch_SAM(im_true, im_fake):
    N = im_true.size()[0]
    C = im_true.size()[1]
    H = im_true.size()[2]
    W = im_true.size()[3]
    Itrue = im_true.clone().reshape(N, C, H*W)
    Ifake = im_fake.clone().reshape(N, C, H*W)
    nom = torch.mul(Itrue, Ifake).sum(dim=1).reshape(N, H*W)
    denom1 = torch.pow(Itrue,2).sum(dim=1).sqrt_().reshape(N, H*W)
    denom2 = torch.pow(Ifake,2).sum(dim=1).sqrt_().reshape(N, H*W)
    sam = torch.div(nom, torch.mul(denom1, denom2)).acos_().reshape(N, H*W)
    sam = sam / np.pi * 180
    sam = torch.sum(sam) / (N*H*W)
    return sam


def weights_init_kaimingNormal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0.2, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)
    elif classname.find('InstanceNorm') != -1:
        nn.init.normal(m.weight.data, 0, 0.01)
        nn.init.constant(m.bias.data, 0.0)

###
class conv_relu_res_relu_block(nn.Module):
    def __init__(self):
        super(conv_relu_res_relu_block, self).__init__()
        self.conv1 = conv3x3(64, 64)
        self.relu1 = nn.ReLU(inplace=True)
        #self.relu1 = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.conv2 = conv3x3(64, 64)
        self.relu2 = nn.ReLU(inplace=True)
        #self.relu2 = nn.LeakyReLU(negative_slope=0.2, inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = torch.add(out,residual) 
        out = self.relu2(out)
        return out
    
#####             resblock

def conv3x3(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=1, padding=1, bias=True)
    
class resblock(nn.Module):
    def __init__(self, block, block_num, input_channel, output_channel):
        super(resblock, self).__init__()

        self.in_channels = input_channel
        self.out_channels = output_channel
        self.input_conv = conv3x3(self.in_channels, out_channels=64)  
        self.conv_seq = self.make_layer(block, block_num)
        self.conv = conv3x3(64, 64)
        self.relu = nn.ReLU(inplace=True)
        self.output_conv = conv3x3(in_channels=64,  out_channels=self.out_channels)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n=m.kernel_size[0]*m.kernel_size[1]*m.out_channels
                m.weight.data.normal_(0,sqrt(2./n))# the devide  2./n  carefully  
                
    def make_layer(self,block,num_layers):
        layers = []
        for i in range(num_layers):
            layers.append(block()) # there is a () 
        return nn.Sequential(*layers)   
    
    def forward(self, x):
       
        out = self.input_conv(x)
        residual = out
        out = self.conv_seq(out)
        out = self.conv(out)
        out = torch.add(out,residual)  
        out = self.relu(out)
        out = self.output_conv(out)
        return out

In [ ]:
## process the data
#data_process_list_n(path=path) 


In [5]:
##. Only image normalization

# Training 
def train(train_data_loader, model, criterion_1, criterion_2, optimizer, iteration, init_lr ,epoch, w1, w2):

    losses = AverageMeter()
    average_MRAE = 0.
    average_RMSE = 0. 
    average_SAM = 0.
    average_SID =0.
    
    num=len(train_data_loader)
    print('num.{}'.format(num))
    #optimizer.zero_grad()
    
    for i, (labels_gt, images_gt) in enumerate(train_data_loader):
    #for i in range(num):
        # Normalize the RGB vectors
        images_gt = torch.as_tensor(images_gt).float().cuda()
        images_gt = images_gt.clamp(1e-8, 1.0 - 1e-8)
        # put it on GPU
        labels_gt = torch.as_tensor(labels_gt).float().cuda() 
        labels_gt = labels_gt.clamp(1e-8, 1.0 - 1e-8)
        ##
        images = images_gt.cuda()
        #
        labels = labels_gt.cuda()
        #
        images = Variable(images, requires_grad=True)
        labels = Variable(labels, requires_grad=True)    
        
        # Decaying Learning Rate
        lr = init_lr #poly_lr_scheduler(optimizer, init_lr, epoch, max_iter=968000, power=1.5) 
        iteration = iteration + 1
        #
        optimizer.zero_grad()
        hs_scaled = model.forward(images)
        #print(hs)
        hs_scaled[hs_scaled.isnan()] = 1e-8
        hs_scaled = hs_scaled.clamp(1e-8, float("Inf"))
        # remove extremes
        hs_scaled[labels==1e-8] = 1e-8
        #
        if w2 ==0:
            loss_1 = criterion_1(labels, hs_scaled)
            loss = loss_1*w1
        else:
            loss_1 = criterion_1(labels, hs_scaled)
            loss_2 = criterion_2(labels, hs_scaled)
            #
            loss = loss_1*w1 + loss_2*w2
        ##
        loss.backward()
        optimizer.step()
        #
        #####
        MRAE = batch_MRAE(labels, hs_scaled)
        RMSE = batch_RMSE(labels, hs_scaled)
        SAM = batch_SAM(labels, hs_scaled)
        SID = batch_SID(labels, hs_scaled)
        #
        average_MRAE    += MRAE.item()
        average_RMSE    += RMSE.item()
        average_SAM    += SAM.item()
        average_SID    += SID.item()
        #
        losses.update(loss.item())
    return losses.avg, average_MRAE/num, average_RMSE/num, average_SAM/num, average_SID/num, iteration, lr

# Validation

def validate(val_loader, model, criterion_1, criterion_2, epoch, w1, w2):
    
    model.eval()
    losses = AverageMeter()
    xxx=0
    num = len(val_loader)
    
    average_MRAE    =0.
    average_RMSE    =0.
    average_SAM    =0.
    average_SID    =0.
    
    for i, (target_gt, input_gt) in enumerate(val_loader):
        # Normalize the RGB vectors
        input_gt = torch.as_tensor(input_gt).float().cuda() 
        input_gt = input_gt.clamp(1e-8, 1.0 - 1e-8)

        input = input_gt.cuda()
        target = torch.as_tensor(target_gt).float().cuda() 
        target = target.clamp(1e-8, 1.0 - 1e-8)
        #
        target = target.cuda(non_blocking=True)
        #
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
    
        # compute hs
        with torch.no_grad():
            hs_scaled = model.forward(input_var)
        # normalize the prediction vector
        hs_scaled[hs_scaled.isnan()] = 1e-8
        hs_scaled = hs_scaled.clamp(1e-8, float("Inf"))
        #
        # remove extremes
        hs_scaled[target_var==1e-8] = 1e-8
        #
        #loss = criterion(target_var, hs_scaled)
        #
        if w2 ==0:
            loss_1 = criterion_1(target_var, hs_scaled)
            loss = loss_1*w1
        else:
            loss_1 = criterion_1(target_var, hs_scaled)
            loss_2 = criterion_2(target_var, hs_scaled)
            #
            loss = loss_1*1 + loss_2*1 
        #####
        MRAE = batch_MRAE(target_var, hs_scaled)
        RMSE = batch_RMSE(target_var, hs_scaled)
        SAM = batch_SAM(target_var, hs_scaled)
        SID = batch_SID(target_var, hs_scaled)
        #
        average_MRAE    += MRAE.item()
        average_RMSE    += RMSE.item()
        average_SAM    += SAM.item()
        average_SID    += SID.item()
        #####
        ## generate a figure compare the reconstructed spectra and ground truth, every epoch
        if epoch%1==0:
            if (i+1)%10==0:
                xxx += 1
                H = target_var.size()[2]
                W = target_var.size()[3]
                real_spectrum = target_var.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                fake_spectrum = hs_scaled.data.cpu().numpy()[0,:,int(H/2),int(W/2)]
                I_spectrum = plot_spectrum(real_spectrum, fake_spectrum, "val_"+str(epoch),i)
                ##
                input_gt_plot = input_gt[0,:,:,:].squeeze().permute(1,2,0).data.cpu().numpy()
                fig, axis = plotwithcolorbar(input_gt_plot, "fake image" )
                fig.savefig(os.path.join(iteration_path, "{}_test_fake_img_{}.png".format(epoch,i)))
                ##
                print("val_num: " + str(xxx))
        #  record loss
        losses.update(loss.item())
    return losses.avg, average_MRAE/num, average_RMSE/num, average_SAM/num, average_SID/num



In [ ]:
###               the best one at the moment
# on cases of combined loss functions, try one first and them add the other one 

##                 the main training part (splitting data to trainng and testing)
loss_type = ["MraeWSid"]#["mrae", "rmse"]
for lt in loss_type:
    iteration_folder = "UAV_MaizeAll_jsZ_{}Loss_BV_tcRGB2msi_HSCNNR_iteration".format(lt)
    model_folder = "UAV_MaizeAll_jsZ_{}Loss_BV_tcRGB2msi_HSCNNR_models".format(lt)
    #loss_type = "mrae" #"mrae"
    iteration_path = os.path.join(os.getcwd(), iteration_folder)
    if not os.path.exists(iteration_path):
        os.makedirs(iteration_path)
    ## new plot function with customed saving path
    
    cudnn.benchmark = True
    ## network architecture
    rgb_features = 3
    hyper_features = 5
    ## load dataset
    print("\nloading dataset ...\n")
    #
    trainDataset = HyperDataset_list_maize_RGB(crop_size=600)  ## here not the training data but the whole data set for this work
    # set the ratio of training and validation set
    validation_split = (1/3)
    
    dataset_len = len(trainDataset) #trainDataset
    indices = list(range(dataset_len))
    
    # Randomly splitting indices:
    val_len = int(np.ceil(validation_split * dataset_len))
    validation_idx = np.random.choice(indices, size=val_len, replace=False)
    train_idx = list(set(indices) - set(validation_idx))
    
    ## Defining the samplers for each phase based on the random indices:
    train_sampler = SubsetRandomSampler(train_idx)
    validation_sampler = SubsetRandomSampler(validation_idx)
    
    # Data Loader (Input Pipeline)
    train_data_loader = DataLoader(dataset=trainDataset,
                                    sampler=train_sampler,
                                    num_workers=1,  
                                    batch_size=1,
                                    shuffle=False,
                                    pin_memory=True)
    
    val_loader = DataLoader(dataset=trainDataset,
                            sampler=validation_sampler,
                            num_workers=1, 
                            batch_size=1,
                            shuffle=False,
                            pin_memory=True)
    # Model               
    model = resblock(conv_relu_res_relu_block, 3, 3,5)
    #
    model.apply(weights_init_kaimingNormal)
    # check whether it is possible to do multi-GPU processing
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    if torch.cuda.is_available():
        model.cuda()
    
    # Parameters, Loss and Optimizer
    start_epoch = 0
    end_epoch = 10000
    init_lr = 1e-4
    
    iteration = 0
    
    ## set the original values for the evaluation
    #
    record_ts_loss= np.inf
    record_ts_MRAE = np.inf
    record_ts_RMSE = np.inf
    record_ts_SAM = np.inf
    record_ts_SID = np.inf #np.inf

    # decide which loss function to use
    #criterion = nn.MSELoss()
    #if lt == "mrae":
    #    criterion = mrae_loss
    #elif lt =="rmse":
    #    criterion = nn.MSELoss()
    #else:
    #    criterion = sid_loss
    
    criterion_1 = mrae_loss #nn.MSELoss()
    criterion_2 = sid_loss #nn.MSELoss()
    #
    w1 = 1
    w2 = 0
    optimizer=torch.optim.Adamax(model.parameters(), lr=init_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
    
    model_path = os.path.join(os.getcwd(), model_folder)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    loss_csv = open(os.path.join(model_path, '{}_loss_{}.csv'.format(lt, str(f))), 'w+')
    
    log_dir = os.path.join(model_path,'{}_train_{}.log'.format(lt,str(f)))
    logger = initialize_logger(log_dir)
    # load the trained model weights if you have
    #resume_file = os.path.join("UAV_MaizeAll_jsZ_MraeWSidLoss_BV_tcRGB2msi_HSCNNR_models", "hscnn_6layer_dim10_ts_loss.pkl")
    resume_file = "" 
    #
    if resume_file:
        print("=> loading checkpoint '{}'".format(resume_file))
        checkpoint = torch.load(resume_file)
        start_epoch = checkpoint['epoch']
        iteration = checkpoint['iter']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        
    for epoch in range(start_epoch + 1, end_epoch): ## specify the number of epochs to run
        
        start_time = time.time()         
        tr_loss, tr_MRAE, tr_RMSE, tr_SAM, tr_SID, iteration, lr = train(train_data_loader, model, criterion_1, criterion_2, optimizer, iteration, init_lr, epoch, w1,w2)
        #
        end_time = time.time() # only record the training time
        epoch_time = end_time - start_time
        print(datetime.datetime.now())
        #lr=init_lr
        ts_loss, ts_MRAE, ts_RMSE,ts_SAM, ts_SID = validate(val_loader, model, criterion_1, criterion_2, epoch, w1,w2)
        #
        #           save a model based on total loss
        if ts_loss < record_ts_loss: 
            record_ts_loss = ts_loss
            save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_loss")
            record_ts_MRAE_loss = ts_MRAE
            record_ts_RMSE_loss = ts_RMSE
            record_ts_SAM_loss = ts_SAM
            record_ts_SID_loss = ts_SID
            lr_loss = init_lr
            print("ts_loss")
            print("Saving total loss", "updated_ts_loss", record_ts_loss,"updated_ts_MRAE", record_ts_MRAE_loss, "updated_ts_RMSE",record_ts_RMSE_loss, "updated_ts_SAM",record_ts_SAM_loss, "updated_ts_SID",record_ts_SID_loss, "LR", lr_loss)
        #           save a model based on MRAE loss
        if ts_MRAE < record_ts_MRAE:  
            record_ts_MRAE = ts_MRAE
            save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_mrae")
            record_ts_loss_MRAE = ts_loss
            record_ts_RMSE = ts_RMSE
            record_ts_SAM_MRAE = ts_SAM
            record_ts_SID_MRAE = ts_SID
            lr_mrae = init_lr
            print("ts_MRAE")
            print("Saving MRAE", "updated_ts_loss", record_ts_loss_MRAE,"updated_ts_MRAE", record_ts_MRAE, "updated_ts_RMSE",record_ts_RMSE, "updated_ts_SAM",record_ts_SAM_MRAE, "updated_ts_SID",record_ts_SID_MRAE, "LR", lr_mrae)
        #           save a model based on SID loss
        if ts_SID < record_ts_SID:  
            record_ts_SID = ts_SID
            save_checkpoint_sp(model_path, epoch, iteration, model, optimizer, "ts_sid")
            record_ts_loss_SID = ts_loss
            record_ts_MRAE_SID = ts_MRAE
            record_ts_RMSE_SID = ts_RMSE
            record_ts_SAM = ts_SAM
            lr_sid = init_lr
            print("ts_SID")
            print("Saving SID", "updated_ts_loss", record_ts_loss_SID,"updated_ts_MRAE", record_ts_MRAE_SID, "updated_ts_RMSE",record_ts_RMSE_SID, "updated_ts_SAM",record_ts_SAM, "updated_ts_SID",record_ts_SID, "LR", lr_sid)
        # print loss
############################################### save model on rice validation 
        #
        # (1) ts_loss evaluation
        print ("Epoch:{}, Iter:{}, Time:{}, learning rate:{}, Train loss:{}, Train MRAE:{}, Train RMSE:{}, Train SAM:{}, Train SID:{}".format(
            epoch, iteration, epoch_time, lr, tr_loss, tr_MRAE, tr_RMSE, tr_SAM, tr_SID))
        print ("Test loss:{}, Test MRAE:{}, Test RMSE:{}, Test SAM:{}, Test SID:{}".format(ts_loss, ts_MRAE, ts_RMSE, ts_SAM, ts_SID))

        logger.info("Epoch{}, Iter{}, Time:{}, learning rate:{}, Train loss:{},Train MRAE:{},Train RMSE:{}, Train SAM:{}, Train SID:{}, Test loss:{},Test MRAE:{},Test RMSE:{}, Test SAM:{}, Test SID:{}".format(
            epoch, iteration, epoch_time, lr, tr_loss, tr_MRAE,tr_RMSE, tr_SAM, tr_SID, ts_loss, ts_MRAE,ts_RMSE,ts_SAM, ts_SID))
        ###
        print("w1: {}, w2: {}".format(w1,w2))
        if (epoch +1)%50 ==0:
            init_lr = init_lr*0.98
            print(("updated learning rate", init_lr))
        if (epoch+1) % 600 ==0:
            clear_output()



loading dataset ...



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:407: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


num.101
2022-08-05 17:16:22.797693
val_num: 1
val_num: 2
val_num: 3
val_num: 4
val_num: 5
ts_loss
Saving total loss updated_ts_loss 0.9661073684692383 updated_ts_MRAE 0.9661070681085774 updated_ts_RMSE 0.1977150437294268 updated_ts_SAM 51.198976180132696 updated_ts_SID 15.669321920357499 LR 0.0001
ts_MRAE
Saving MRAE updated_ts_loss 0.9661073684692383 updated_ts_MRAE 0.9661070681085774 updated_ts_RMSE 0.1977150437294268 updated_ts_SAM 51.198976180132696 updated_ts_SID 15.669321920357499 LR 0.0001
ts_SID
Saving SID updated_ts_loss 0.9661073684692383 updated_ts_MRAE 0.9661070681085774 updated_ts_RMSE 0.1977150437294268 updated_ts_SAM 51.198976180132696 updated_ts_SID 15.669321920357499 LR 0.0001
fold: 0, Epoch:1, Iter:101, Time:14.014204978942871, learning rate:0.0001, Train loss:1.1264142877984755, Train MRAE:1.0768019261926707, Train RMSE:0.21460151067464658, Train SAM:33.4704206296713, Train SID:4.601732792535631
Test loss:0.9661073684692383, Test MRAE:0.9661070681085774, Test RMSE:0.